In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load datasets
bird_essays = pd.read_csv('/content/drive/MyDrive/data/BARD/BARD_essay.csv')  # columns: index, prompts, BARD
gpt_essays = pd.read_csv('/content/drive/MyDrive/data/GPT/ChatGPT_essay.csv')    # columns: index, prompts, responses
human_essays = pd.read_csv('/content/drive/MyDrive/data/Human/human_essay_1.csv') # columns: index, essays

# Prepare the texts and add source labels
bird_essays['text'] = bird_essays['BARD']  # Adjust the column name accordingly
gpt_essays['text'] = gpt_essays['responses']  # Adjust the column name accordingly
human_essays['text'] = human_essays['essays']  # Adjust the column name accordingly

# Add source labels
bird_essays['source'] = 1  # BARD
gpt_essays['source'] = 0    # GPT
human_essays['source'] = 2   # Human

# Combine datasets
essays_data = pd.concat([bird_essays[['text', 'source']], gpt_essays[['text', 'source']], human_essays[['text', 'source']]], ignore_index=True)

# Clean the text
essays_data['text'] = (
    essays_data['text'].str.replace(r'http\S+|www\S+|https\S+', '', case=False)  # Remove URLs
    .str.replace(r'<.*?>', '', case=False)  # Remove HTML tags
    .str.replace(r'\s+', ' ', regex=True)  # Remove extra whitespaces
    .str.lower()  # Convert to lowercase
)

# Handle NaN values: replace NaN with an empty string
essays_data['text'] = essays_data['text'].fillna('')

# Prepare features and labels
X = essays_data['text']
y = essays_data['source']

# Tokenization with DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_texts, test_texts, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)  # Reduced sequence length to 256
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)

# Define dataset class
class EssaysDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = EssaysDataset(train_encodings, y_train.values)
test_dataset = EssaysDataset(test_encodings, y_test.values)

# Load pre-trained DistilBERT model for classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Set up DataLoader with a smaller batch size
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # Reduced batch size to 4
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Mixed precision setup
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop with gradient accumulation and mixed precision
accumulation_steps = 4  # Accumulate gradients over 4 steps

model.train()
for epoch in range(3):  # You can increase the number of epochs for better performance
    optimizer.zero_grad()
    for i, batch in enumerate(train_loader):
        with autocast():
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss / accumulation_steps  # Normalize the loss by accumulation steps
            scaler.scale(loss).backward()

        # Perform optimization step every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

# Evaluation
model.eval()
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        y_pred.extend(predictions.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(classification_report(y_test, y_pred, target_names=['GPT', 'BARD', 'Human']))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-1-30a4b187a946>:80: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
<ipython-input-1-30a4b187a946>:92: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_t

Accuracy: 0.9860
Precision: 0.9860
Recall: 0.9860
F1-score: 0.9860
              precision    recall  f1-score   support

         GPT       0.90      0.90      0.90        40
        BARD       0.90      0.90      0.90        40
       Human       1.00      1.00      1.00       493

    accuracy                           0.99       573
   macro avg       0.93      0.93      0.93       573
weighted avg       0.99      0.99      0.99       573

